In [259]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [269]:
#get law groups
driver = webdriver.Chrome(executable_path='/mnt/c/charin_projects/thai-law/chromedriver.exe')
driver.get('https://www.krisdika.go.th/web/guest/thai-code-annotated')
soup = BeautifulSoup(driver.page_source)
driver.close()

law_groups = pd.DataFrame([i.text for i in soup.find_all('a',class_='ksdk-theme-bg-third-color')])
law_groups['law_group'] = law_groups[0].map(lambda x: x.split('(')[0][:-1])
law_groups['nb_laws'] = law_groups[0].map(lambda x: int(x.split('(')[1][:-1]))
law_groups['nb_pages'] = np.ceil(law_groups['nb_laws']/10).astype(int)
law_groups = law_groups.drop(0,1)
# law_groups.to_csv('data/v0.3/law_groups.csv',index=False)
law_groups

,law_group,nb_laws,nb_pages
0,กษัตริย์,12,2
1,การเงิน การคลัง และวิธีการงบประมาณ,146,15
2,การเมืองการปกครอง,269,27
3,ขนส่งและคมนาคม,59,6
4,คณะกรรมการกฤษฏีกา,2,1
5,คนต่างด้าว,56,6
6,ครอบครัว และมรดก,69,7
7,ความมั่นคง และการรักษาความสงบเรียบร้อย,53,6
8,ความเท่าเทียมระหว่างเพศ,1,1
9,คุ้มครองผู้บริโภค,56,6


In [270]:
def get_law_urls(law_group, nb_pages):
    #open list page
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get('https://www.krisdika.go.th/web/guest/thai-code-annotated')
    #click law group
    link = driver.find_element_by_partial_link_text(law_group)
    link.click()
    
    #check if max pagination button appeared
    try:
        element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.LINK_TEXT, str(nb_pages)))
        )
    except:
        print('Max pagination button not found')
    
    #get law _urls
    laws = []
    law_urls = []
    for nb_page in tqdm(range(1, nb_pages+1)):
        link = driver.find_element_by_link_text(str(nb_page))
        link.click()
        soup = BeautifulSoup(driver.page_source)
        laws+=[i.text for i in soup.find_all('li', class_='thca-list-law-name')]
        law_urls+=[i.find_all('li')[-1].find('a').get('href') for i in soup.find_all('ul', class_='thca-list-icon')]
        
    #close driver
    driver.close()
    
    #summarize to df
    df = pd.DataFrame({'title':laws, 'law_url':law_urls})
    df['sysid'] = df.law_url.map(lambda x: x.split('=')[-2].split('&')[0])
    df['law_group'] = law_group
    return df

In [272]:
dfs = []
for row in tqdm(law_groups.itertuples(index=False)):
    print(row[0])
    df = get_law_urls(row[0],row[2])
    dfs.append(df)

กษัตริย์



การเงิน การคลัง และวิธีการงบประมาณ



การเมืองการปกครอง



ขนส่งและคมนาคม



คณะกรรมการกฤษฏีกา


...


เครื่องราชอิสริยาภรณ์



เครื่องแบบ เสื้อครุย



เอกสิทธิ์และความคุ้มกัน



แรงงาน



โทรคมนาคม และการสื่อสาร


In [273]:
law_url_df = pd.concat(dfs)
law_url_df

,title,law_url,sysid,law_group
0,พระราชกำหนดโอนอัตรากำลังพลและงบประมาณบางส่วนขอ...,https://www.krisdika.go.th/librarian/get?sysid...,843218,กษัตริย์
1,พระราชบัญญัติระเบียบบริหารราชการในพระองค์ พ.ศ....,https://www.krisdika.go.th/librarian/get?sysid...,775999,กษัตริย์
2,รัฐธรรมนูญแห่งราชอาณาจักรไทย,https://www.krisdika.go.th/librarian/get?sysid...,774606,กษัตริย์
3,ประมวลกฎหมายอาญา (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,443287,กษัตริย์
4,ประมวลกฎหมายวิธีพิจารณาความอาญา (ฉบับ Update ล...,https://www.krisdika.go.th/librarian/get?sysid...,570066,กษัตริย์
...,...,...,...,...
7,พระราชบัญญัติจดแจ้งการพิมพ์ พ.ศ. 2550,https://www.krisdika.go.th/librarian/get?sysid...,568883,โทรคมนาคม และการสื่อสาร
8,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,319040,โทรคมนาคม และการสื่อสาร
9,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,302303,โทรคมนาคม และการสื่อสาร
10,พระราชบัญญัติกองทุนพัฒนาสื่อปลอดภัยและสร้างสรร...,https://www.krisdika.go.th/librarian/get?sysid...,725393,โทรคมนาคม และการสื่อสาร


In [323]:
#designate law types
law_types = ['รัฐธรรมนูญ', 'พระราชบัญญัติ', 'พระราชกำหนด','ประมวลกฎหมาย', 'ประมวลรัษฎากร','ประกาศ','คำสั่ง','พระธรรมนูญ']

def get_law_type(law_name, law_types=law_types, max_char=13):
    for l in law_types:
        if l in law_name[:max_char]:
            return l
    return 'others'

law_url_df['law_type'] = law_url_df.title.map(get_law_type)
law_url_df.law_type.value_counts()

พระราชบัญญัติ    2012
พระราชกำหนด        81
ประมวลกฎหมาย       54
รัฐธรรมนูญ         32
ประกาศ             21
ประมวลรัษฎากร      11
คำสั่ง              3
พระธรรมนูญ          1
Name: law_type, dtype: int64

In [324]:
# law_url_df.to_csv('data/v0.3/law_url_df.csv',index=False)
law_url_df = pd.read_csv('data/v0.3/law_url_df.csv')
law_url_df

,title,law_url,sysid,law_group,law_type
0,พระราชกำหนดโอนอัตรากำลังพลและงบประมาณบางส่วนขอ...,https://www.krisdika.go.th/librarian/get?sysid...,843218,กษัตริย์,พระราชกำหนด
1,พระราชบัญญัติระเบียบบริหารราชการในพระองค์ พ.ศ....,https://www.krisdika.go.th/librarian/get?sysid...,775999,กษัตริย์,พระราชบัญญัติ
2,รัฐธรรมนูญแห่งราชอาณาจักรไทย,https://www.krisdika.go.th/librarian/get?sysid...,774606,กษัตริย์,รัฐธรรมนูญ
3,ประมวลกฎหมายอาญา (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,443287,กษัตริย์,ประมวลกฎหมาย
4,ประมวลกฎหมายวิธีพิจารณาความอาญา (ฉบับ Update ล...,https://www.krisdika.go.th/librarian/get?sysid...,570066,กษัตริย์,ประมวลกฎหมาย
...,...,...,...,...,...
2210,พระราชบัญญัติจดแจ้งการพิมพ์ พ.ศ. 2550,https://www.krisdika.go.th/librarian/get?sysid...,568883,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ
2211,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,319040,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ
2212,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,302303,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ
2213,พระราชบัญญัติกองทุนพัฒนาสื่อปลอดภัยและสร้างสรร...,https://www.krisdika.go.th/librarian/get?sysid...,725393,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ


In [276]:
law_groups.nb_laws.sum()

2215

In [293]:
v02 = pd.read_csv('data/v0.2/law.csv')
v02 = v02[v02.sysid!='None'].reset_index(drop=True)
v02['sysid'] = v02.sysid.map(lambda x: int(x)) 
v02

,sysid,title,txt
0,726890,กฎบัตรสหประชาชาติ ซึ่งรับตกลงกัน ณ เมืองซานฟาน...,\n\n\n\n\n\n\n\n\nกฎบัตรสหประชาชาติ\n\n\n\n\n\...
1,301085,กฎบัตรสหประชาชาติ ค.ศ. 1945,\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n \n \n \n ...
2,740727,ประกาศกระทรวงมหาดไทย เรื่อง การออกหนังสือผ่านแ...,\n\n\n\n\n\n\n\n\nประกาศกระทรวงมหาดไทย\n\n\n\n...
3,497860,ประกาศการมีผลใช้บังคับของความตกลงระหว่างรัฐบาล...,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nประกาศ\r\n...
4,490191,ประกาศ การมีผลใช้บังคับของความตกลง ระหว่าง รัฐ...,\n\n\n\n\n\n\n\n\nประกาศ\n\n\n\n\n\n\nประกาศ\n...
...,...,...,...
42749,771111,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42750,771127,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42751,771129,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42752,771099,ข้อบัญญัติองค์การบริหารส่วนตำบลสว่างแดนดิน เรื...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...


In [295]:
law_url_df.merge(v02,how='left').txt.isna().sum()

1312

In [297]:
v02

,sysid,title,txt
0,726890,กฎบัตรสหประชาชาติ ซึ่งรับตกลงกัน ณ เมืองซานฟาน...,\n\n\n\n\n\n\n\n\nกฎบัตรสหประชาชาติ\n\n\n\n\n\...
1,301085,กฎบัตรสหประชาชาติ ค.ศ. 1945,\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n \n \n \n ...
2,740727,ประกาศกระทรวงมหาดไทย เรื่อง การออกหนังสือผ่านแ...,\n\n\n\n\n\n\n\n\nประกาศกระทรวงมหาดไทย\n\n\n\n...
3,497860,ประกาศการมีผลใช้บังคับของความตกลงระหว่างรัฐบาล...,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nประกาศ\r\n...
4,490191,ประกาศ การมีผลใช้บังคับของความตกลง ระหว่าง รัฐ...,\n\n\n\n\n\n\n\n\nประกาศ\n\n\n\n\n\n\nประกาศ\n...
...,...,...,...
42749,771111,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42750,771127,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42751,771129,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42752,771099,ข้อบัญญัติองค์การบริหารส่วนตำบลสว่างแดนดิน เรื...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...


In [300]:
law_url_df.sysid.value_counts()

774606    32
567410    15
443287    13
617709    11
564433     9
          ..
701147     1
701143     1
701141     1
803507     1
305153     1
Name: sysid, Length: 868, dtype: int64

พระราชบัญญัติ    2012
พระราชกำหนด        81
ประมวลกฎหมาย       54
รัฐธรรมนูญ         32
ประกาศ             21
ประมวลรัษฎากร      11
คำสั่ง              3
พระธรรมนูญ          1
Name: law_type, dtype: int64